### Recursos

Problemas interesantes para Aprendizaje por refuerzo
 * Gymnasium: https://gymnasium.farama.org/environments/box2d/
 * Solución del Lunar Lander con DQN: https://shiva-verma.medium.com/solving-lunar-lander-openaigym-reinforcement-learning-785675066197
 * Otra solución: https://wingedsheep.com/lunar-lander-dqn/ y https://github.com/wingedsheep/blog/blob/main/lunar-lander-dqn/lunar_lander_dqn_blog.ipynb
 * The Nature of Code: https://youtu.be/lu5ul7z4icQ
 * Librería para neuroevolución: https://pypi.org/project/nevopy/

## Instalación

!pip install gymnasium  
!pip install gymnasium[box2d] 

## Acciones adicionales

### En macos

pip uninstall swig  
xcode-select -—install (si no se tienen ya)  
pip install swig  
pip install 'gymnasium[box2d]' # en zsh hay que poner las comillas  

### en Windows

Si da error, se debe a la falta de la versión correcta de Microsoft Visual C++ Build Tools, que es una dependencia de Box2D. Para solucionar este problema, puede seguir los siguientes pasos:  
 * Descargar Microsoft Visual C++ Build Tools desde https://visualstudio.microsoft.com/visual-cpp-build-tools/.
 * Dentro de la app, seleccione la opción "Herramientas de compilación de C++" para instalar.
 * Reinicie su sesión en Jupyter Notebook.
 * Ejecute nuevamente el comando !pip install gymnasium[box2d] en la línea de comandos de su notebook.

In [1]:
# prueba lunar lander por humano
# apartado A de la práctica

import gymnasium as gym

env = gym.make("LunarLander-v2", render_mode="rgb_array")

import numpy as np
import pygame
import gymnasium.utils.play

lunar_lander_keys = {
    (pygame.K_UP,): 2,
    (pygame.K_LEFT,): 1,
    (pygame.K_RIGHT,): 3,
}
gymnasium.utils.play.play(env, zoom=3, keys_to_action=lunar_lander_keys, noop=0)

In [1]:
# agente deliberativo
# Apartado B de la práctica

# observaciones [x, y, vx, vy, ang, vang, pataiz, parader]
# acciones [nada, derecho, central, izquierdo]

def policy (observation):
    if <condiciones>:
        print('⬅︎',end='')
        return 1
    if <condiciones>:
        print('⬆︎',end='')
        return 3
    if <condiciones>:
        print('⮕',end='')
        return 2
    print('–',end='')
    return 0

In [ ]:
# neuroevolución
# Apartado C de la práctica

# construir modelo
model = ***
model.from_chromosome(ch)

# pasar al modelo los pesos del mejor cromosoma obtenido con SALGA

# definir política
def policy (observation):
    s = model.forward(observation)
    action = np.argmax(s)
    return action

In [ ]:
# prueba lunar lander por agente

import gymnasium as gym

env = gym.make("LunarLander-v2", render_mode="human")

def run ():
    #observation, info = env.reset(seed=42)
    observation, info = env.reset()
    ite = 0
    racum = 0
    while True:
        action = policy(observation)    
        observation, reward, terminated, truncated, info = env.step(action)
        
        racum += reward

        if terminated or truncated:
            r = (racum+200) / 500
            print(racum, r)
            return racum
    
#run()

In [ ]:
N = 10
r = 0
for _ in range(N):
    r += run()
    
print(r/N)

In [ ]:
# walker

env = gym.make("BipedalWalker-v3", render_mode="human") # hardcore=True
...